In [6]:
"""
Since the output of at any CVT points for HDPE is not available,
we will not carry out evaluating metrics on the generated virtual samples,
as the way that have done for magical_sinus benchmarking dataset. Instead,
we indirectly confirm the quality of the generated samples by showing the
ability of our RegCGAN on capturing P(y|x).
"""


import importlib

import dataset, metrics, plotting, config, network
from models import reg_cgan_model
import numpy as np
import random

importlib.reload(network)
importlib.reload(dataset)
importlib.reload(metrics)
importlib.reload(plotting)
importlib.reload(config)
importlib.reload(reg_cgan_model)

<module 'models.reg_cgan_model' from '/Users/zhongsheng/Documents/GitWorkspace/RegCGAN/models/reg_cgan_model.py'>

In [8]:
import os

dataset_config = config.DatasetConfig(scenario="hdpe")

assert(dataset_config.scenario == "magical_sinus"
      or dataset_config.scenario == "hdpe")
fig_dir = f"../figures/{dataset_config.scenario}"

try:
    os.mkdir(fig_dir)
    print(f"Directory {fig_dir} created ")
except FileExistsError:
    print(f"Directory {fig_dir} already exists replacing files in this notebook")

Directory ../figures/hdpe already exists replacing files in this notebook


In [9]:
exp_config = config.Config(
    model=config.ModelConfig(activation="elu", lr_gen=0.0001, lr_disc=0.0001,
                             optim_gen="Adam", optim_disc="Adam", z_input_size=5),
    training=config.TrainingConfig(n_epochs=10000, batch_size=100, n_samples=500),
    dataset=dataset_config,
    run=config.RunConfig(save_fig=1)
)

TypeError: __new__() got an unexpected keyword argument 'n_samples'

In [10]:
# Set random seed
np.random.seed(exp_config.model.random_seed)
random.seed(exp_config.model.random_seed)

from tensorflow import set_random_seed
set_random_seed(exp_config.model.random_seed)

NameError: name 'exp_config' is not defined

In [11]:
X_train, y_train, X_valid, y_valid = dataset.get_dataset(scenario=exp_config.dataset.scenario,
                                                                         seed=exp_config.model.random_seed)

NameError: name 'exp_config' is not defined

In [12]:
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler()
X_train_scaled = X_scaler.fit_transform(X_train)
X_valid_scaled = X_scaler.fit_transform(X_valid)

y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1, 1))
y_valid_scaled = y_scaler.fit_transform(y_valid.reshape(-1, 1))

NameError: name 'X_train' is not defined

## Gaussian Process

In [13]:
import GPy

variance = 0.1
length = 1

kernel = GPy.kern.RBF(input_dim=15, variance=variance, lengthscale=length)
gpr = GPy.models.GPRegression(X_train_scaled, y_train_scaled, kernel)

run_hyperopt_search = True
if run_hyperopt_search:
    gpr.optimize(messages=True)

NameError: name 'X_train_scaled' is not defined

## Construct CGAN model

In [ ]:
regcgan = reg_cgan_model.RegCGAN(exp_config)
d_loss_err, d_loss_true, d_loss_fake, g_loss_err, g_pred, g_true = regcgan.train(X_train, y_train,
                                                                              epochs=exp_config.training.n_epochs,
                                                                              batch_size=exp_config.training.batch_size,
                                                                              verbose=True)

In [ ]:
plotting.plot_training_curve(d_loss_err, d_loss_true, d_loss_fake, g_loss_err, g_pred, g_true, fig_dir, exp_config.run.save_fig)

## Generate pairs of virtual samples

In [5]:
from os.path import basename
X_cvt = np.load(f"{fig_dir}/{basename(fig_dir)}_cvt_samples.npy")
X_cvt_scaled = X_scaler.transform(X_cvt)

NameError: name 'np' is not defined

In [12]:
ypred_recgan_cvt = regcgan.predict(X_cvt_scaled)
ypred_gp_cvt, cov_cvt = gpr.predict(X_cvt_scaled)

In [13]:
ypred_recgan_cvt = y_scaler.inverse_transform(ypred_recgan_cvt)
ypred_gp_cvt = y_scaler.inverse_transform(ypred_gp_cvt)

In [15]:
X_cvt_paird = np.c_[X_cvt, ypred_recgan_cvt]

In [15]:
from os.path import basename
np.save(f"{fig_dir}/{basename(fig_dir)}_CVT_samples_paired.npy", X_cvt_paird)


NameError: name 'X_cvt_paird' is not defined